In [1]:
from argparse import ArgumentParser

import numpy as np
import sympy as sp
from lal import GreenwichMeanSiderealTime

from gwbench import Network, injections_CBC_params_redshift, M_of_Mc_eta, f_isco_Msolar

/var/folders/z7/zgpk7rr930x6gm2p_cg3h7kh0000gn/T/ipykernel_93652/2763875321.py:5: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  from lal import GreenwichMeanSiderealTime


In [2]:
np.set_printoptions(linewidth=200)

parser = ArgumentParser()
parser.add_argument('--inj', type = int, help = 'Injection ID in [0,100).', default = 0)
parser.add_argument('--derivs', type = str, help = 'Specify wich differentiation method to use: [num, sym].', default = 'num')

parser.add_argument("-f", required=False)

_StoreAction(option_strings=['-f'], dest='f', nargs=None, const=None, default=None, type=None, choices=None, required=False, help=None, metavar=None)

In [3]:
############################################################################
### User Choices
############################################################################

# choose between numeric or symbolic derivatives ['num', 'sym']
derivs = parser.parse_args().derivs

# choose injection id
inj_id = parser.parse_args().inj

# user's choice: waveform to use
# wf_model_name = 'tf2'
#wf_model_name = 'tf2_tidal'
wf_model_name = 'lal_bbh'
#wf_model_name = 'lal_bns'


# the lal_... waveform models are wrappers to call frequency domain waveform from lalsuite
# and thus need further specification of the approximant to use
if   wf_model_name == 'tf2':       wf_other_var_dic = None
elif wf_model_name == 'tf2_tidal': wf_other_var_dic = None
elif wf_model_name == 'lal_bbh':   wf_other_var_dic = {'approximant':'TaylorF2Ecc'}
elif wf_model_name == 'lal_bns':   wf_other_var_dic = {'approximant':'IMRPhenomD_NRTidalv2'}

# user defined waveform model, defined by the files specified in the dictionary (although if 1 is always true and no user defined waveform
# path is taken but if we want to define our own wavefrom then make if 0 )
# only the 'np' one is needed when using numeric derivatives
if 1: user_waveform = None
else:
    wf_model_name    = 'tf2_user'
    wf_other_var_dic = None
    user_waveform   = {'np': '../gwbench/wf_models/tf2_np.py', 'sp':'../gwbench/wf_models/tf2_sp.py'}

In [4]:
# example detector location defined by user (keep it empty if the detector tech is not custom defined )
# user_locs = {'user-loc':{'longitude': 0.0, 'latitude': 0.0, 'arm_azimuth':0.0, 'which_arm':'y', 'shape':'L'}}
user_locs = {}

# example detector psd defined by the user (keep it empty if the detector tech is not custom defined )
# (edit this if using different psd, path can be put as absolute path)(for PSD keep it False but the data files available in noise
# curve folder are all ASD, so keep is_asd=True)
# user_psds = {'user-tec':{'psd_file':'/Users/mlaxman/Documents/gwbench/gwbench/noise_curves/a_plus.txt', 'is_asd':True}}
user_psds = {}


# user's choice: with respect to which parameters to take derivatives for the Fisher analysis
if 'tidal' in wf_model_name or 'bns' in wf_model_name: deriv_symbs_string = 'Mc eta DL tc phic e0 ra dec psi iota'
# else: deriv_symbs_string = 'Mc eta tc phic chi1z chi2z e0'
else: deriv_symbs_string = 'Mc eta DL tc phic e0 ra dec psi iota'

# user's choice: convert derivatives to cos or log for specific variables
conv_cos = ('iota', 'dec')
conv_log = ('Mc','DL')

# if symbolic derivatives, take from generate_lambdified_functions.py
# if numeric  derivatives, user's decision
use_rot = 0
# 1 for True, 0 for False

# calculate SNRs, error matrices, and errors only for the network (If 1, will calculate the mentioned values only for network otherwise
# will calculate it for detectors as well as network)
only_net = 1

# number of cores to use for parallelize of the calc_det_responses_derivs
# = None for no parallelization, = 2,3,4,... to allocate N cores (even numbers preferred)
num_cores = None

In [5]:
# options for numeric derivative calculation
if derivs == 'num':
    # user's choice: switch particular partial derivatives to be analytical, options = [DL,tc,phic,ra,dec,psi]
    # otherwise set to None
    # ana_deriv_symbs_string = 'DL tc phic ra dec psi' # If any of the variable is in cos or log in above code, do the same here as well
    # ana_deriv_symbs_string = 'sp.log(DL) tc phic ra sp.cos(dec) psi'
    ana_deriv_symbs_string = None
    # np.cos(dec)
   

    # choose numdifftools parameters for numerical derivatives
    step      = 1e-9
    method    = 'central'
    order     = 2

    # only relevant for symbolic derivatives
    gen_derivs = None

# options for symbolic derivative calculation
elif derivs == 'sym':

    # user's choice: switch particular partial derivatives to be analytical, options = [DL,tc,phic,ra,dec,psi]
    # otherwise set to None
    ana_deriv_symbs_string = None

    # choose numdifftools parameters for numerical derivatives
    step      = None
    method    = None
    order     = None

    # tell the code to generate symbolic derivatives as needed (turned on this tutorial)
    # the recommendation is to precompute them externally and load them for large-scale runs
    gen_derivs = True


In [6]:
# user's choice to generate injection parameters
# if 'tidal' in wf_model_name or 'bns' in wf_model_name:
#     mmin      = 0.8
#     mmax      = 3
#     chi_lo    = -0.05
#     chi_hi    = 0.05
# else:
#     mmin      = 20
#     mmax      = 30
#     chi_lo    = 0.0
#     chi_hi    = 0.0

# cosmo_dict = {'zmin':0, 'zmax':0.2, 'sampler':'uniform_comoving_volume_inversion'}
# mass_dict  = {'dist':'uniform', 'mmin':mmin, 'mmax':mmax}
# # the default waveforms above are non-precessing, hence dim=1, set dim=3 for precessing waveforms like 'IMRPhenomPv2' or 'IMRPhenomPv2_NRTidalv2'
# spin_dict  = {'dim':1, 'geom':'cartesian', 'chi_lo':chi_lo, 'chi_hi':chi_hi}

# # Flag indicating whether to apply redshift to the mass parameters (redshifted : bool).
# redshifted = 1
# # Number of injections to generate (default: 10)
# num_injs   = 10
# # Seed for the random number generator (default: None)
# seed       = 29378
# file_path  = None

# # Generate parameters [(redshifted) masses, spins, luminosity distance, redshift, inclination, right ascension, declination, polarization angle
# # for compact binary coalescence (CBC) injections.
# injections_data = injections_CBC_params_redshift(cosmo_dict,mass_dict,spin_dict,redshifted,num_injs,seed,file_path)


In [7]:
############################################################################
### injection parameters
############################################################################

inj_params = {
    'Mc'    : 6.723034342261816,
    'e0'    : 0.3,
    'eta'   : 0.2222222222222222,
    'chi1x' : 0.,
    'chi1y' : 0.,
    'chi1z' : 0.,
    'chi2x' : 0.,
    'chi2y' : 0.,
    'chi2z' : 0.,
    'DL'    : 500,
    'tc'    : 0.,
    'phic'  : 0.,
    'iota'  : np.random.uniform(0, np.pi),
    'ra'    : np.random.uniform(0, 2 * np.pi),
    'dec'   : np.random.uniform(0, np.pi),
    'psi'   : np.random.uniform(0, 2 * np.pi),
    'gmst0' : 0.,
    'z'     : 0.1051,
    }


if 'tidal' in wf_model_name or 'bns' in wf_model_name:
    inj_params['lam_t']       = 600.
    inj_params['delta_lam_t'] = 0.

# print(injections_data[0][99])

print('injections parameter: ', inj_params)
print()

injections parameter:  {'Mc': 6.723034342261816, 'e0': 0.3, 'eta': 0.2222222222222222, 'chi1x': 0.0, 'chi1y': 0.0, 'chi1z': 0.0, 'chi2x': 0.0, 'chi2y': 0.0, 'chi2z': 0.0, 'DL': 500, 'tc': 0.0, 'phic': 0.0, 'iota': 1.0866837689154896, 'ra': 2.826610196992887, 'dec': 2.7878413091523324, 'psi': 2.9016580232381197, 'gmst0': 0.0, 'z': 0.1051}



In [8]:
############################################################################
### Network specification
############################################################################
# 'user-tec_user-loc' is for custom define detector and its location
network_spec = ['aLIGO_L']


print('network spec: ', network_spec)
print()

f_lo = 10
# f_hi = f_isco_Msolar(M_of_Mc_eta(inj_params['Mc'],inj_params['eta']))
f_hi = 268
df   = 2.**-4.5
f    = np.arange(f_lo,f_hi+df,df)


print('f_lo:', f_lo, '   f_hi:', f_hi, '   df:', df)
print()

network spec:  ['aLIGO_L']

f_lo: 10    f_hi: 268    df: 0.04419417382415922



In [12]:
############################################################################
### Single Network GW Benchmarking
############################################################################

# initialize Network and do general setup
net = Network(network_spec, logger_name='CSU', logger_level='INFO')
# pass all the needed and optional variables
net.set_net_vars(wf_model_name=wf_model_name, wf_other_var_dic=wf_other_var_dic, user_waveform=user_waveform,
                 f=f, inj_params=inj_params, deriv_symbs_string=deriv_symbs_string,
                 conv_cos=conv_cos, conv_log=conv_log, use_rot=use_rot,
                 user_locs=user_locs, user_psds=user_psds, ana_deriv_symbs_string=ana_deriv_symbs_string)
# net.set_net_vars(wf_model_name=wf_model_name, wf_other_var_dic=wf_other_var_dic, user_waveform=user_waveform,
#                  f=f, inj_params=inj_params, deriv_symbs_string=deriv_symbs_string,
#                  conv_log=conv_log, use_rot=use_rot,
#                  user_locs=user_locs, user_psds=user_psds, ana_deriv_symbs_string=ana_deriv_symbs_string)
# start the actual analysis
net.calc_errors(only_net=only_net, derivs=derivs, step=step, method=method, order=order, gen_derivs=gen_derivs, num_cores=num_cores)

### for completeness here are the steps involved, but since gwbench-0.7.4 the code will perform the necessary
### internally as needed
# '''
# net.setup_ant_pat_lpf_psds()
# if derivs == 'num':
#     net.calc_det_responses_derivs_num(step=step, method=method, order=order, num_cores=num_cores)
# elif derivs = 'sym':
#     # generation of lambdified derivatives used to be handles outside and is still recommended for speed reasons
#     # refer to example_script/generate_lambdified_functions.py
#     net.load_det_responses_derivs_sym(gen_derivs=True)
#     net.calc_det_responses_derivs_sym(num_cores=num_cores)
# net.calc_snrs(only_net=only_net)
# net.calc_errors(only_net=only_net)
# '''

2024-08-02 20:07:17,991 - CSU - INFO : PSDs, antenna patterns, and LPFs loaded.
2024-08-02 20:07:17,992 - CSU - INFO : Calculate numeric derivatives of detector responses.
2024-08-02 20:07:17,993 - CSU - INFO :    aLIGO_L
2024-08-02 20:07:18,283 - CSU - INFO : Numeric derivatives of detector responses calculated.
2024-08-02 20:07:18,283 - CSU - INFO : SNRs calculated.
2024-08-02 20:07:18,284 - CSU - INFO : Calculate errors (Fisher & cov matrices).
2024-08-02 20:07:18,284 - CSU - INFO :    aLIGO_L
2024-08-02 20:07:18,306 - CSU - INFO : Sky areas calculated.
2024-08-02 20:07:18,306 - CSU - INFO : Errors calculated.


In [13]:
############################################################################
### Print results
############################################################################

# net.print_detectors()
# net.print_network()

# print('Fisher analysis done.')

In [14]:
print("SNR:", net.snr)
print("errs:", net.errs)
# print("Error in e0:", net.errs['e0'])
# print("Error in e0:", net.errs['e0']/0.3)

SNR: 8.10403799582788
errs: {'log_Mc': 0.002493614, 'eta': 0.009039281, 'log_DL': 290.55334, 'tc': 134.49762, 'phic': 6240.2363, 'e0': 0.002469196, 'ra': 6314.0913, 'cos_dec': 2206.3916, 'psi': 3721.9656, 'cos_iota': 2594.4917, 'sky_area_90': 1783712118991.113}


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define lower limit, upper limit, and step size for e0 values
e0_lower = 0.001
e0_upper = 0.30
step_size = 0.002

# Generate e0 values from lower limit to upper limit with the given step size
e0_values = np.arange(e0_lower, e0_upper+step_size, step_size)

# List to store e0 values and corresponding errors
e0_errors = []

# Loop over the generated e0 values
for e0_value in e0_values:
    # Update e0 value in inj_params
    inj_params['e0'] = e0_value
    
    # Print current e0 value
    print(f'Computing error for e0 = {e0_value}')
    
    # Initialize Network and do general setup
    net = Network(network_spec, logger_name='CSU', logger_level='INFO')
    # pass all the needed and optional variables
    net.set_net_vars(wf_model_name=wf_model_name, wf_other_var_dic=wf_other_var_dic, user_waveform=user_waveform,
                     f=f, inj_params=inj_params, deriv_symbs_string=deriv_symbs_string,
                     conv_cos=conv_cos, conv_log=conv_log, use_rot=use_rot,
                     user_locs=user_locs, user_psds=user_psds, ana_deriv_symbs_string=ana_deriv_symbs_string)
    # start the actual analysis
    net.calc_errors(only_net=only_net, derivs=derivs, step=step, method=method, order=order, gen_derivs=gen_derivs, num_cores=num_cores)

    # Get error in e0
    e0_error = net.errs['e0']
    
    # Print error in e0
    print("Error in e0:", e0_error)
    
    # Append e0 value and corresponding error to the list
    e0_errors.append([e0_value, e0_error])

# Print the 2D list of e0 values and errors
# print("e0 values and corresponding errors:")
# for e0_value, e0_error in e0_errors:
#     print(f'e0: {e0_value}, Error: {e0_error}')


In [ ]:
# np.savetxt('favata_fig1_bbh3_fhi189p5_dfm1_stepm11.txt', e0_errors, fmt='%f %f')
# np.savetxt('favata_fig1_bbh2_fhi241_dfm2p8_stepm11.txt', e0_errors, fmt='%f %f')
np.savetxt('favata_fig1_bbh1_fhi241_dfm2p8_stepm11.txt', e0_errors, fmt='%f %f')